In [1]:
#Look up at the commit message first

In [2]:
import os
from dotenv import load_dotenv

import pandas as pd
from sqlalchemy import create_engine, inspect

In [3]:
load_dotenv()

True

In [4]:
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

In [5]:
DATABASE_URI = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"

In [6]:
engine = create_engine(DATABASE_URI)

In [7]:
connection = engine.connect()

In [8]:
def select(sql):
    return pd.read_sql(sql, connection)

In [9]:
inspect(engine).get_table_names()

['active_clients', 'cards', 'offers']

# Задание №1	
Описание данных: таблица ACTIVE_CLIENTS содержит ежемесячный срез клиентов банка, которые совершали какие-либо транзакции в данном месяце. Атрибуты: отчетный месяц (report_month) и идентификатор клиента (client_id). Считаем, что клиент «оттек» из банка в месяце N, если в месяце N он активен (присутствует в таблице ACTIVE_CLIENTS) и не активен в месяцы N+1, N+2, N+3. | REPORT_MONTH | CLIENT_ID  |
|--------------|------------|
| 2018-01-01   | 1847982357 |
| 2018-01-01   | 938475     |
| 2018-02-01   | 1847982357 |
| 2018-02-01   | 6789998    |
| 2018-03-01   | 6790000З
…	…

Задание: вывести количество активных клиентов на каждый месяц; долю клиентов, которые «оттекли» в каждом месяце.


In [10]:
sql = '''SELECT *   
         FROM active_clients
         LIMIT 1'''

select(sql)

,report_month,client_id
0,2018-01-01,938475


In [11]:
sql = '''
      WITH inactive as (
                        SELECT YEAR(report_month) AS Year,
                                MONTH(report_month) AS Month,
                                CLIENT_ID,
                                CASE WHEN LEAD(REPORT_MONTH, 1) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH ASC) IS NULL OR
                                          LEAD(REPORT_MONTH, 2) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH ASC) IS NULL OR
                                          LEAD(REPORT_MONTH, 3) OVER (PARTITION BY CLIENT_ID ORDER BY REPORT_MONTH ASC) IS NULL 
                                     THEN 1
                                     ELSE 0
                                     END AS inactive_count
                
                         FROM active_clients
                        )
                         
        SELECT Year,
               Month,
               COUNT(DISTINCT CLIENT_ID) as ActiveCount,
               SUM(inactive_count) as InectiveCount,
               ROUND(SUM(inactive_count) / COUNT(DISTINCT CLIENT_ID), 2) AS InactiveRate
        
        FROM inactive
        GROUP BY Year, Month
      '''

select(sql)

,Year,Month,ActiveCount,InectiveCount,InactiveRate
0,2018,1,4,0.0,0.00
1,2018,2,4,0.0,0.00
2,2018,3,5,0.0,0.00
3,2018,4,4,1.0,0.25
4,2018,5,6,1.0,0.17
5,2018,6,4,0.0,0.00
6,2018,7,3,0.0,0.00
7,2018,8,5,1.0,0.20
8,2018,9,6,0.0,0.00
9,2018,10,5,1.0,0.20


# Задание №2
Описание данных:  таблица OFFERS содержит информацию о кредитных предложениях клиентов банка. У каждого предложения есть идентификатор (offer_id), дата начала действия (offer_start_date), дата окончания действия (offer_expiration_date). Предложение считается действующим на дату dt, если dt находится между offer_start_date и offer_expiration_date для данного предложения.

Задание: вывести количество действующих предложений на каждый день 2018 года. 
| OFFER_ID | OFFER_START_DATE | OFFER_EXPIRATION_DATE |
|----------|------------------|-----------------------|
| 83942    | 2017-12-01       | 2018-02-01            |
| 94859    | 2018-05-03       | 2018-10-19            |
| …        | …                | …                     |
…	…	…


In [12]:
sql = '''
      SELECT *
      FROM OFFERS
      LIMIT 1
      '''

select(sql)

,offer_id,offer_start_date,offer_expiration_date
0,10002,2018-10-01,2019-03-01


In [13]:
sql = '''
      WITH RECURSIVE cte as (
                              SELECT '2018-01-01' AS Day
                              UNION ALL
                              SELECT DATE_ADD(Day, INTERVAL 1 DAY)
                              FROM cte
                              WHERE DAY < '2018-12-31'
                            )
                            
      SELECT cte.Day,
             count(o.OFFER_ID) as OfferCount
             
      FROM cte LEFT JOIN OFFERS o 
      ON cte.Day BETWEEN o.OFFER_START_DATE AND OFFER_EXPIRATION_DATE
      
      GROUP BY cte.Day
      ORDER BY cte.Day
      '''

select(sql)

,Day,OfferCount
0,2018-01-01,12
1,2018-01-02,12
2,2018-01-03,12
3,2018-01-04,12
4,2018-01-05,12
...,...,...
360,2018-12-27,10
361,2018-12-28,10
362,2018-12-29,10
363,2018-12-30,10


# Задание №3
Описание данных:  Таблица CARDS содержит информацию о всех картах клиентов банка. Атрибуты: идентификатор клиента (client_id),  идентификатор карты (card_id),  дата выдачи карты (open_date), дата закрытия карты (close_date), тип карты – дебетовая (DC) или кредитная (СС) (card_type). 

Задание: выбрать из всех работающих на дату 1.09.2018 дебетовых карт клиента ту, которая была выдана последней. Формат выдачи – client_id, card_id.
| CLIENT_ID | CARD_ID     | OPEN_DATE   | CLOSE_DATE  | CARD_TYPE |
|-----------|-------------|-------------|-------------|-----------|
| 1232110   | 49582985729 | 2019-01-12  | NULL        | DC        |
| 234235    | 48574092749 | 2017-03-29  | 2018-09-01  | CC        |
| …         | …           | …           | …           | …         |

In [14]:
sql = '''
      SELECT *
      FROM CARDS
      LIMIT 1
      '''

select(sql)

,client_id,card_id,open_date,close_date,card_type
0,4456221,47583984722,2018-07-10,None,DC


In [15]:
sql = '''
      WITH cte AS (
                   SELECT *,
                          ROW_NUMBER() OVER (PARTITION BY CLIENT_ID ORDER BY OPEN_DATE DESC) AS rn
                   FROM CARDS
                   WHERE card_type = 'DC' AND
                         open_date <= '2018-09-01' AND
                         (close_date is NULL OR close_date > '2018-09-01')
                  )
                  
      SELECT client_id,
             card_id
      FROM cte
      WHERE rn = 1
      '''

select(sql)

,client_id,card_id
0,1234001,48573984799
1,2345002,48573984800
2,3413412,48573984711
3,4456221,47583984722
4,4567004,48573984802
5,6543211,48573984788
6,6789006,48573984804
7,7546123,48573984755
8,7890007,48573984805
9,8234512,48573984766


In [16]:
connection.close()